Ratings 
- Documentation is quite good. It's beginner-friendly, and provides various kinds of tutorials.
- Access UI: localhost/dashboard

In [7]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [ ]:
encoder = SentenceTransformer("all-MiniLM-L6-v2")

In [10]:
# define where to store embeddings (currently in my local memory)
client = QdrantClient(url="http://localhost:6333")

In [11]:
client.set_model("sentence-transformers/all-MiniLM-L6-v2")
# comment this line to use dense vectors only
client.set_sparse_model("prithivida/Splade_PP_en_v1")

In [ ]:
# create a collection

client.create_collection(
    collection_name="test2",
    # vectors_config=models.VectorParams(
    #     size=encoder.get_sentence_embedding_dimension(),  # Vector size is defined by used model
    #     distance=models.Distance.COSINE,
    # ),
    vectors_config=client.get_fastembed_vector_params(),
    sparse_vectors_config=client.get_fastembed_sparse_vector_params(),
)

In [23]:
import json

json_file = open('test_data.json', encoding='utf-8')
test_data = json.load(json_file)

korean = []
english = []
metadata = []

for line in test_data:
    korean.append(line.pop("korean"))
    english.append(line.pop("english"))
    metadata.append(line)

In [ ]:

# encode
client.add(
    collection_name="test2",
    documents=english,
    metadata=metadata,
    parallel=0,  # Use all available CPU cores to encode data. 
    # Requires wrapping code into if __name__ == '__main__' block
)

In [ ]:
from tqdm import tqdm
# upload
client.add(
    collection_name="startups",
    documents=english,
    metadata=metadata,
    ids=tqdm(range(len(english))),
)

In [27]:
client.upload_points(
    collection_name="test",
    points=[
        models.PointStruct(
            id=idx, vector=encoder.encode(doc["english"]).tolist(), payload=doc
        )
        for idx, doc in enumerate(test_data)
        
      
    ],
)

## Hybrid search
the hybrid search class will need 3 things: 
1) models to convert the query into a vector, 
2) the Qdrant client to perform search queries, 
3) fusion function to re-rank dense and sparse search results.